In [1]:
import numpy as np
import csv 
import time


In [2]:
np.random.seed(1234)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def randomize() -> None:
    np.random.seed(time.time())

In [4]:
# Hyperparameter 설정
RND_MEAN = 0 
RND_STD = 0.0030 # 가중치 파라미터를 초기화 할 때 이용함

LEARNING_RATE = 0.001

In [5]:
def load_abalone_dataset() -> None:
    with open("/Users/hyeonjin/workspace/dataset/abalone/abalone.data.csv") as f: # abalone 데이터셋이 존재하는 경로를 따라 csv 모듈로 로드
        reader = csv.reader(f)
        rows = []
        for idx, row in enumerate(reader):
            rows.append(row) # 데이터를 차례로 rows에 append

    global data, input_cnt, output_cnt # 전역 변수 data, input_cnt, output_cnt 생성
    input_cnt, output_cnt = 10, 1 # input_cnt 가 10인 이유는 성별을 나타내는 원핫 벡터 3개 + 7개의 데이터, output_cnt는 현미경으로 관찰한 고리 수, 라벨 데이터
    data = np.zeros([len(rows), input_cnt+output_cnt]) # numpy 모듈로 4177행, 11열 모양을 갖는 영행렬 생성

    for idx, row in enumerate(rows): # csv로 얻은 데이터를 numpy로 생성한 벡터에 parsing
        # 성별에 따른 원핫 벡터 (모든 행에대해 0, 1, 2열) 생성
        if row[0] == 'I': data[idx, 0] = 1
        if row[0] == 'M': data[idx, 1] = 1
        if row[0] == 'F': data[idx, 2] = 1
        # 나머지 속성은 성별 뒤에 붙임
        data[idx, 3:] = row[1:]

In [6]:
def init_model() -> None:
    global weight, bias, input_cnt, output_cnt # 전역 변수로 wieght, bias 생성, load_abalone_dataset에서 생성한 input_cnt, output_cnt 사용
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt]) # RND_MEAN, RND_STD를 이용해 input_cnt * output_cnt 형태의 파라미터 생성
    bias = np.zeros([output_cnt])

In [7]:
def forward_postproc(output, y):
    # 회귀 분석에 맞추어 output과 y로부터 손실 함수 loss 계산
    diff = output - y # 오차
    square = np.square(diff) # 제곱
    loss = np.mean(square) # 평균
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape

    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

def backprop_postproc_oneline(G_loss, diff):
    return 2 * diff / np.prod(diff.shape)

def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

In [8]:
def forward_neuralnet(x):
    global weight, bias # init_model에서 선언한 wight, bias를 이용
    output = np.matmul(x, weight) + bias # XW + b 수행
    return output, x

def backprop_neuralnet(G_output, x): # 역전파 처리 함수, 순전파 출력 output에 대한 손실 기울기 G_output 
    global weight, bias # init_model에서 선언한 wegith, bias를 이용
    g_output_w = x.transpose() # x를 이용해 x와 output 사이의 부분 기울기 g_output_w를 구함

    G_w = np.matmul(g_output_w, G_output) # 부분 기울기와 손실 기울기를 이용해 weight 성분의 손실 기울기 구함
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [9]:
def run_train(x, y):
    # 순전파 과정 수행
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [10]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx # load_abalone_data 함수에서 생성한 data 전역 변수
    shuffle_map = np.arange(data.shape[0]) # abalone data의 행 길이 만큼의 1차원 리스트 생성
    np.random.shuffle(shuffle_map) # 리스트를 섞음
    step_count = int(data.shape[0] * 0.8) // mb_size # 전체 데이터의 80퍼센트를 미니 배치 사이즈로 나누어 step_count 정의
    test_begin_idx = step_count * mb_size # 테스트 데이터의 시작 인덱스 번호 생성
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt # load_abalone_data와 arrage_data에서 생성한 변수들 사용
    test_data = data[shuffle_map[test_begin_idx:]] # 테스트 데이터의 시작 위치를 이용해 test_data 생성
    return test_data[:, :-output_cnt], test_data[:, -output_cnt] # 10개의 속성값이 x, 마지막 열이 label 데이터

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt # load_abalone_data, arrange_data에서 생성한 변수들 사용
    if nth == 0: # 각 에폭의 첫 번째 호출일때 학습 데이터 부분에 대한 부분적인 순서를 섞어 에폭마다 다른 순서로 학습이 수행되게 함
        np.random.shuffle(shuffle_map[:test_begin_idx])
    
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:] # 10개의 속성값이 x, 마지막 열이 label 데이터

In [11]:
def train_and_test(epoch_count:int, mb_size:int, report:int) -> None:
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
        
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print(f"Epoch {epoch+1} loss={np.mean(losses):5.3f} accuracy={np.mean(accs):5.3f}/{acc:5.3f}.")
    
    final_acc = run_test(test_x, test_y)
    print(f"\nFinal Test: final accuracy = {final_acc:5.3f}")

In [12]:
def abalone_exec(epoch_count:int=10, mb_size:int=10, report:int=1) -> None:
    load_abalone_dataset() # 데이터셋 읽어들이는 load_abalone_dataset 실행
    init_model() # 모델의 파라미터들을 초기화 하는 init_model 실행
    train_and_test(epoch_count, mb_size, report) # 학습 및 평가 과정을 수행하는 train_and_test 함수 호출

In [13]:
abalone_exec()

Epoch 1 loss=33.875 accuracy=0.557/ -inf.
Epoch 2 loss=8.226 accuracy=0.820/ -inf.
Epoch 3 loss=7.582 accuracy=0.812/ -inf.
Epoch 4 loss=7.475 accuracy=0.808/ -inf.
Epoch 5 loss=7.395 accuracy=0.810/ -inf.
Epoch 6 loss=7.328 accuracy=0.808/ -inf.
Epoch 7 loss=7.269 accuracy=0.808/ -inf.
Epoch 8 loss=7.217 accuracy=0.808/ -inf.
Epoch 9 loss=7.175 accuracy=0.810/ -inf.
Epoch 10 loss=7.135 accuracy=0.809/ -inf.

Final Test: final accuracy =  -inf


/var/folders/km/75btkg954xl31kcf44f3wzf40000gn/T/ipykernel_63011/2853326565.py:25: RuntimeWarning: divide by zero encountered in true_divide
  mdiff = np.mean(np.abs((output - y)/y))


In [14]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100, mb_size=100, report=20)

Epoch 20 loss=5.804 accuracy=0.825/ -inf.
Epoch 40 loss=5.259 accuracy=0.834/ -inf.
Epoch 60 loss=5.056 accuracy=0.837/ -inf.
Epoch 80 loss=4.950 accuracy=0.838/ -inf.
Epoch 100 loss=4.910 accuracy=0.840/ -inf.

Final Test: final accuracy =  -inf


/var/folders/km/75btkg954xl31kcf44f3wzf40000gn/T/ipykernel_63011/2853326565.py:25: RuntimeWarning: divide by zero encountered in true_divide
  mdiff = np.mean(np.abs((output - y)/y))
